In [133]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [134]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df_lending = pd.read_csv("Resources/lending_data.csv")

# Review the DataFrame
print(df_lending.shape)
df_lending.head()

(77536, 8)


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [135]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df_lending['loan_status']

# Separate the X variable, the features
X = df_lending.drop(columns='loan_status')

In [136]:
# Review the y variable Series
y[:5]


0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [137]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [138]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [139]:

# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [140]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs',
                                random_state=1)
classifier

LogisticRegression(random_state=1)

In [141]:
# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [142]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [143]:
from sklearn.metrics import accuracy_score
# Print the balanced_accuracy score of the model
accuracy_score(y_test, predictions)

0.9918489475856377

In [144]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [145]:
# Print the classification report for the model
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {accuracy_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,18663,102
Actual 1,56,563


Accuracy Score : <function accuracy_score at 0x000001CF20E430D0>
Classification Report
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** The model demonstrates stronger performance in predicting the "0" class compared to the "1" class. Nevertheless, considering the accuracies achieved for both classes, it can be considered a reliable predictive model. The model achieves a perfect accuracy score of 1.00 when predicting the "0" class, signifying its ability to correctly classify healthy loans with complete accuracy. However, in the case of the "1" class, which represents risky loans, the model achieves an accuracy rate of 86%.

The recall metric gauges the model's capacity to accurately recognize the specific classes within the dataset. The model attains a 100% recall rate for the "0" class, indicating that it correctly identifies these instances consistently. For the "1" class, the model exhibits a 90% recall rate, implying that it accurately identifies high-risk loans in the dataset 90% of the time.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [146]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversamp_model = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resample, y_resample = random_oversamp_model.fit_resample(X_train, y_train)

In [147]:
# Count the distinct values of the resampled labels data
y_resample.value_counts()

0    56271
1    56271
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [148]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr_model_2= LogisticRegression(random_state=1)

# Fit the model using the resampled training data
train_model = lr_model_2.fit(X_resample,y_resample)

# Make a prediction using the testing data
testing_prediction2= lr_model_2.predict(X_test)

In [149]:
results = pd.DataFrame({"Prediction": testing_prediction2, "Actual": y_test})
results.head(15)


,Prediction,Actual
60914,0,0
36843,0,0
1966,0,0
70137,0,0
27237,0,0
40013,0,0
43107,0,0
61988,0,0
57437,0,0
46757,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [150]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, testing_prediction2)

0.9936781215845847

In [151]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, testing_prediction2))

[[18649   116]
 [    4   615]]


In [152]:
# Print the classification report for the model
print(classification_report(y_test, testing_prediction2, target_names=('Healthy Loans','High Risk Loans')))

                 precision    recall  f1-score   support

  Healthy Loans       1.00      0.99      1.00     18765
High Risk Loans       0.84      0.99      0.91       619

       accuracy                           0.99     19384
      macro avg       0.92      0.99      0.95     19384
   weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:**  The logistic regression model, which was trained using oversampled data, exhibits a commendable performance in predicting both the "healthy loan" and "high-risk loan" labels. It achieves a substantial accuracy score of 0.99, indicating its ability to accurately predict the majority of instances for both types of loans - healthy and risky.

The precision value for class 0, corresponding to healthy loans, is a flawless 1.00. This signifies that every instance predicted as a "healthy loan" is indeed a true "healthy loan."

In the case of the "high-risk loan" label (label 1), the precision stands at 0.84. This suggests that approximately 84% of the instances predicted as "high-risk loans" are indeed true "high-risk loans."

For the label "0," the precision remains at a perfect 1.00, implying that nearly all the instances predicted as "healthy loans" accurately belong to the category of true "healthy loans."

The recall value for this category is impressively high at 0.99, underscoring the model's efficacy in identifying the majority of true "high-risk loans."

The model's strong performance, evident from its high accuracy and well-balanced precision and recall metrics for both classes, establishes it as a robust predictor capable of distinguishing between "healthy loans" and "high-risk loans" within the dataset.